# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

587

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:
#establish base url for API call
url = 'http://api.openweathermap.org/data/2.5/weather?'
units = 'metric'

#create query url for loop
query_url = f'{url}appid={weather_api_key}&units={units}&q='

In [ ]:
test_response = requests.get(query_url + 'houston').json()
test_response['main']

In [ ]:
# create empty lists to store data for dataframe
latitude = []
temp = []
humidity = []
cloudiness = []
wind_speed = []

In [6]:
# loop through cities to collect currect weather data
for city in cities:
    # Create search query, make request and store in json
    response_json = requests.get(query_url + city).json()
    
    # Try to grab the height and mass of characters if they are available in the Star Wars API
    try:
        latitude.append(test_response['coord']['lat'])
        temp.append(test_response['main']['temp'])
        #mass.append(response_json['results'][0]['mass'])
        print("city added...")
        
    # Handle exceptions for a cities not found or errors
    except KeyError:
        cities.remove(city)
        print("City not found..")
        pass

city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city added...
city a

KeyboardInterrupt: 

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
# Create DataFrame
weather_data = pd.DataFrame({
    'City': cities,
    'Latitude': latitude
    #'Temperature (F)': temp,
    #'Humidity (%)': humidity,
    #'Cloudiness (%)': cloudiness,
    #'Wind Speed (mph)': wind_speed
})

weather_data
#weather_data = weather_data.set_index('City')

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression